In [1]:
import network as layer
import torch.nn as nn

nc = 3
nz = 512

In [2]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.utils.data
lr     = 0.0002
beta1  = 0.5   
beta2  = 0.99     
imageSize = 128
batchSize = 32

outf = "./celeba_result/"
des_dir = "./celeba/"

dataset = dset.ImageFolder(root=des_dir,
                           transform=transforms.Compose([
                               transforms.CenterCrop(224),
                               transforms.Resize(imageSize),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size= batchSize,
                                         shuffle=True)

In [3]:
from custom_layers import *

class Generator(nn.Module):
    def make_dense(self, k_in, k_growth, n, options):
        layers = []
        for i in range(n):
            layers.append(Dense(layer.conv(k_in, k_growth, 3, 1, 1, **options)))
            k_in += k_growth
        return nn.Sequential(*layers)
    
    def __init__(self):
        super(Generator, self).__init__()
        options = {'leaky':True, 'bn':False, 'wn':True, 'pixel':True}
        layers = []
        k_growth = 12
        layers.append(layer.conv(nz, 256, 4, 1, 3, **options))
        
        layers.append(self.make_dense(256, k_growth, 3, options))
        layers.append(layer.conv(292, 256, 1, 1, 0, **options))
        layers.append(layer.conv(256, 256, 3, 1, 1, **options))
        # 4 x 4
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(self.make_dense(256, k_growth, 3, options))
        layers.append(layer.conv(292, 256, 1, 1, 0, **options))
        layers.append(layer.conv(256, 256, 3, 1, 1, **options))
        # 8 x 8
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(self.make_dense(256, k_growth, 3, options))
        layers.append(layer.conv(292, 256, 1, 1, 0, **options))
        layers.append(layer.conv(256, 256, 3, 1, 1, **options))
        # 16 x 16
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(self.make_dense(256, k_growth, 3, options))
        layers.append(layer.conv(292, 128, 1, 1, 0, **options))
        layers.append(layer.conv(128, 128, 3, 1, 1, **options))
        # 32 x 32
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(self.make_dense(128, k_growth, 3, options))
        layers.append(layer.conv(164, 64, 1, 1, 0, **options))
        layers.append(layer.conv(64, 64, 3, 1, 1, **options))
        # 64 x 64
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(self.make_dense(64, k_growth, 3, options))
        layers.append(layer.conv(100, 32, 1, 1, 0, **options))
        layers.append(layer.conv(32, 32, 3, 1, 1, **options))
        # 128 x 128
        layers.append(layer.conv(32, nc, 1, 1, 0, leaky=True, bn=False, wn=True, pixel=True, only=True))
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)
    
netG = Generator()

In [4]:
class Discriminator(nn.Module):
    def make_dense(self, k_in, k_growth, n, options):
        layers = []
        for i in range(n):
            layers.append(Dense(layer.conv(k_in, k_growth, 3, 1, 1, **options)))
            k_in += k_growth
        return nn.Sequential(*layers)
    
    def __init__(self):
        super(Discriminator, self).__init__()
        
        options = {'leaky':True, 'bn':False, 'wn':True, 'pixel':False, 'gdrop':True}
        
        layers = []
        layers.append(layer.conv(nc, 16, 1, 1, 0, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(layer.conv(16, 16, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(layer.conv(16, 32, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(nn.AvgPool2d(kernel_size=2))
        
        layers.append(layer.conv(32, 32, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(layer.conv(32, 64, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(nn.AvgPool2d(kernel_size=2))
        
        layers.append(layer.conv(64, 64, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(layer.conv(64, 128, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(nn.AvgPool2d(kernel_size=2))
        
        layers.append(layer.conv(128, 128, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(layer.conv(128, 256, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(nn.AvgPool2d(kernel_size=2))
        
        layers.append(layer.conv(256, 256, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(layer.conv(256, 512, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(nn.AvgPool2d(kernel_size=2))
        
        layers.append(minibatch_std_concat_layer())
        layers.append(layer.conv(513, 512, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(layer.conv(512, 512, 4, 1, 0, leaky=True, bn=False, wn=True, pixel=False, gdrop=True))
        layers.append(layer.linear(512, 1, sig=False, wn=True))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)
    
netD = Discriminator()

In [5]:
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision.utils as vutils

In [6]:
criterion = nn.MSELoss()

input = torch.FloatTensor(batchSize, 3, imageSize,imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)

label_real = torch.FloatTensor(batchSize)
label_fake = torch.FloatTensor(batchSize)

netD.cuda()
netG.cuda()
criterion.cuda()
input, label_real, label_fake = input.cuda(), label_real.cuda(), label_fake.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
label_real.resize_(batchSize, 1).fill_(1)
label_fake.resize_(batchSize, 1).fill_(0)
label_real = Variable(label_real)
label_fake = Variable(label_fake)
print()

In [7]:
netD.load_state_dict(torch.load(outf + 'netD_epoch_042.pth'))
netG.load_state_dict(torch.load(outf + 'netG_epoch_042.pth'))

In [8]:
fixed_noise = Variable(fixed_noise)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=0.00005, betas=(beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr=0.0001, betas=(beta1, beta2))
schedulerD = optim.lr_scheduler.MultiStepLR(optimizerD, milestones=[4, 7, 11, 17], gamma=0.87)
schedulerG = optim.lr_scheduler.MultiStepLR(optimizerG, milestones=[4, 7, 11, 17], gamma=0.87)
result_dict = {}
loss_D,loss_G,score_D,score_G1,score_G2 = [],[],[],[],[]

In [ ]:
import numpy as np
_d_ = None
def add_noise(x, d_fake):
    global _d_
    if _d_ is not None:
        _d_ = _d_ * 0.9 + torch.mean(d_fake).data[0] * 0.1
        strength = 0.2 * max(0, _d_ - 0.5)**2
        z = np.random.randn(*x.size()).astype(np.float32) * strength
        z = Variable(torch.from_numpy(z)).cuda()
        return x + z
    else:
        _d_ = 0.0
        return x


In [ ]:
niter = 100
d_fake_save = None
for epoch in range(niter):
    schedulerD.step()
    schedulerG.step()
    if epoch < 43:
        continue
    for i, data in enumerate(dataloader, 0):

        # train D
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)

        real_cpu = real_cpu.cuda()
        input.resize_as_(real_cpu).copy_(real_cpu)

        inputv = Variable(input)
        inputv = add_noise(inputv, d_fake_save)
        
        d_real = netD(inputv)
        d_real_mean = d_real.data.mean()
        
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        
        d_fake_save = d_fake = netD(fake.detach())
        d_fake_mean = d_fake.data.mean()
        
        loss_d = criterion(d_real, label_real) + criterion(d_fake, label_fake)
        loss_d.backward()
        optimizerD.step()
        
        # train G
        netG.zero_grad()
        d_fake = netD(fake)
        loss_g = criterion(d_fake, label_real.detach())
        loss_g.backward()
        optimizerG.step()

        if i%250 == 0:
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03dstep_%04d.png' % (outf, epoch, i),
                    normalize=True)
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f'
                  % (epoch, niter, i, len(dataloader),
                     loss_d.data[0], loss_g.data[0], d_real_mean, d_fake_mean))
            
    torch.save(netG.state_dict(), '%s/netG_epoch_%03d.pth' % (outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%03d.pth' % (outf, epoch))

[43/100][0/6330] Loss_D: 0.1647 Loss_G: 0.7477 D(x): 0.8466 D(G(z)): 0.2126
[43/100][250/6330] Loss_D: 0.3390 Loss_G: 0.4950 D(x): 0.6796 D(G(z)): 0.3228
[43/100][500/6330] Loss_D: 0.3881 Loss_G: 0.5182 D(x): 0.6463 D(G(z)): 0.3603
[43/100][750/6330] Loss_D: 0.3216 Loss_G: 0.4454 D(x): 0.7265 D(G(z)): 0.3679
[43/100][1000/6330] Loss_D: 0.3811 Loss_G: 0.4784 D(x): 0.5347 D(G(z)): 0.2873
[43/100][1250/6330] Loss_D: 0.2865 Loss_G: 0.5600 D(x): 0.8613 D(G(z)): 0.3481
[43/100][1500/6330] Loss_D: 0.2970 Loss_G: 0.6008 D(x): 0.6394 D(G(z)): 0.2188
[43/100][1750/6330] Loss_D: 0.3311 Loss_G: 0.5029 D(x): 0.7469 D(G(z)): 0.3473
[43/100][2000/6330] Loss_D: 0.2757 Loss_G: 0.5429 D(x): 0.7649 D(G(z)): 0.2958
[43/100][2250/6330] Loss_D: 0.2320 Loss_G: 0.5457 D(x): 0.7595 D(G(z)): 0.2908
[43/100][2500/6330] Loss_D: 0.3165 Loss_G: 0.6064 D(x): 0.6812 D(G(z)): 0.2724
[43/100][2750/6330] Loss_D: 0.3073 Loss_G: 0.8576 D(x): 0.8316 D(G(z)): 0.2490
[43/100][3000/6330] Loss_D: 0.3311 Loss_G: 0.6148 D(x): 0.